<a href="https://colab.research.google.com/github/gabryellesoares/NLP/blob/main/Modelagem_de_T%C3%B3picos_com_LDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Modelagem de tópicos com LDA

Gabryelle Soares

116210883

In [ ]:
import pandas as pd
import numpy as np
import altair as alt

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

import re
from time import time

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
data = pd.read_csv('https://raw.githubusercontent.com/nazareno/palavras-nas-letras/master/letras-ptbr-rock-grande.csv')

In [ ]:
letras = data.query('Artist != "Rosa de Saron"').copy(deep = True)  

letras['titulo_limpo'] = letras['SName'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
letras.drop_duplicates(subset ="titulo_limpo", 
                       keep = 'first', inplace = True)

count = letras['Lyric'].str.split().str.len()
letras = letras[count > 50]

print('Letras: {}'.format(letras.shape[0]))
print('\nPor artista: ')
letras['Artist'].value_counts()[0:20]

Letras: 5277

Por artista: 


Rita Lee                   245
Erasmo Carlos              206
Lulu Santos                202
Engenheiros do Hawaii      178
Charlie Brown Jr           177
Titãs                      174
Capital Inicial            174
Raul Seixas                169
Os Paralamas do Sucesso    161
Biquini Cavadão            157
Barão Vermelho             144
Velhas Virgens             131
Fresno                     129
Jota Quest                 125
Skank                      122
Ira!                       119
Pato Fu                    118
Detonautas                 108
PG                         108
Nx Zero                    103
Name: Artist, dtype: int64

## Pré-processamento

In [ ]:
stop_words = set(stopwords.words("portuguese"))
stop_words.update(['que', 'até', 'esse', 
                    'essa', 'pro', 'pra',
                    'oi', 'lá', 'blá', 'bb', 
                    'bbm', 'abm', 'cbm', 
                    'dbm', 'dos', 
                    'ltda', 'editora']), 

clean_lyrics = []
for w in range(len(letras.Lyric)):
  lyric = letras['Lyric'].iloc[w]

  # remove special characters and digits
  lyric  = re.sub("(\\d|\\W)+|\w*\d\w*"," ",lyric )
  lyric = ' '.join(s for s in lyric.split() if (not any(c.isdigit() for c in s)) and len(s) > 2)
  clean_lyrics.append(lyric)

clean_lyrics[0:5]

['Enquanto ela estiver aqui Ainda haverá amor Com ela estou feliz com ela enfrento dor Não adianta fugir não adianta chorar dia ela sumir nada mais irá sobrar Sonhar viver todo dia agradecer rezar pra você ser última morrer Podem tentar atingir Podem mandar pra onde for Enquanto ela estiver aqui vida ainda tem valor Sonhar viver todo dia agradecer rezar pra você ser última morrer Uma gota você vale mais que tudo Quando solução não pode resolver fico com você esperança fico com você Sonhar viver todo dia agradecer rezar pra você ser última morrer Sonhar viver todo dia agradecer rezar pra você ser última morrer esperança Esperança',
 'deu luz fez sonhar Chorou por mim fez amar Foi pra mim Foi tudo mais pouco enfim Canto quando sinto sua falta Rezo pelo bem sua alma queria que você vivesse mais vida levou deixou aqui que você ensinou que faz feliz Que Deus bom lugar guarde pra mim quando for minha vez encontro por Fale mais Quero ouvir Onde está Não vejo mas pra sentir Foi pra mim Foi tud

In [ ]:
#COUNT vectorizer
tf_vectorizer = CountVectorizer(
        min_df = 30,
        max_df = 0.5,
        max_features = 10000,
        stop_words = stop_words, 
        ngram_range = (1,2)
  )

#transform
vec_text = tf_vectorizer.fit_transform(clean_lyrics)

#returns a list of words.
words = tf_vectorizer.get_feature_names()

print(vec_text.shape)
print(len(words))

(5277, 1712)
1712


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
words[0:10]

['aberta',
 'abismo',
 'abra',
 'abraça',
 'abraçar',
 'abraço',
 'abre',
 'abrigo',
 'abrir',
 'abriu']

## Encontrando tópicos

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation

In [ ]:
def print_top_words(model, feature_names, n_top_words):
  for topic_idx, topic in enumerate(model.components_):
    print("\n--\nTopic #{}: ".format(topic_idx + 1))
    message = ", ".join([feature_names[i]
                          for i in topic.argsort()[:-n_top_words - 1:-1]])
    print(message)
  print()

def display_topics(W, H, feature_names, documents, no_top_words, no_top_documents):
    for topic_idx, topic in enumerate(H):
        print("\n--\nTopic #{}: ".format(topic_idx + 1))
        print(", ".join([feature_names[i]
                for i in topic.argsort()[:-no_top_words - 1:-1]]).upper())
        top_d_idx = np.argsort(W[:,topic_idx])[::-1][0:no_top_documents]
        for d in top_d_idx: 
          doc_data = letras[['Artist', 'SName']].iloc[d]
          print('{} - {} : \t{:.2f}'.format(doc_data[1], doc_data[0], W[d, topic_idx]))

In [ ]:
lda = LatentDirichletAllocation(n_components=5, # k
                                learning_method='online', # 'online' equivale a minibatch no k-means
                                random_state=0)

t0 = time()

lda.fit(vec_text)
doc_topic_matrix = lda.transform(vec_text)

print("done in %0.3fs." % (time() - t0))

done in 28.186s.


In [ ]:
print('Matriz documento-tópicos:' + str(doc_topic_matrix.shape))
print('Matriz tópicos-termos:' + str(lda.components_.shape))

Matriz documento-tópicos:(5277, 5)
Matriz tópicos-termos:(5, 1712)


In [ ]:
display_topics(doc_topic_matrix,
               lda.components_, 
               words,
               letras,
               10, 
               8)


--
Topic #1: 
VAI, NINGUÉM, FAZ, NUNCA, BEM, VOU, AQUI, FAZER, SEI, TUDO
Senhor da Razão - Aliados : 	0.99
O Que Eu Faço - Aliados : 	0.99
Ilusão da Perfeição - Aliados : 	0.99
Vou Mudar - Aliados : 	0.95
Nem Sempre Se Pode Ser Deus - Titãs : 	0.95
Noviças do Vício - Rita Lee : 	0.95
333 - Charlie Brown Jr : 	0.94
Histórias Antigas - Aliados : 	0.92

--
Topic #2: 
VOU, MUNDO, VIDA, DIA, SEI, NADA, ONDE, TODO, TUDO, SEMPRE
Sorrindo - Aliados : 	0.99
Contrastes da Vida - Charlie Brown Jr : 	0.99
Cantar - Raul Seixas : 	0.99
Eu Vou - Aliados : 	0.99
A Estrada - Titãs : 	0.99
Me Chama - Biquini Cavadão : 	0.98
Sweet Begônia - Engenheiros do Hawaii : 	0.98
Sem Saber Pra Onde Ir - Ira! : 	0.98

--
Topic #3: 
AMOR, QUERO, SOL, VEM, CÉU, DEUS, CORAÇÃO, VIDA, SER, LUZ
Pedra Murano (Victor Rice Dub Mix #2) - Nx Zero : 	0.99
Pedra Murano - Nx Zero : 	0.99
Pedra Murano (Victor Rice Dub Mix #1) - Nx Zero : 	0.99
Muito Além - Ira! : 	0.99
Todo o Amor Que Houver Nessa Vida - Cazuza : 	0.99
Todo Amor

In [ ]:
main_topic = []
mt_prob = []
for l in range(len(letras['Artist'])):
  main_topic.append(doc_topic_matrix[l,:].argmax() + 1)
  mt_prob.append(doc_topic_matrix[l,:].max())

letras = letras.assign(main_topic = main_topic, main_topic_prob = mt_prob)

In [ ]:
topico = 1
pd.options.display.max_colwidth = 300
letras[letras['main_topic'] == topico].sort_values('main_topic_prob', ascending = False)[['main_topic_prob','Artist', 'Lyric']].head(20).sample(10)

,main_topic_prob,Artist,Lyric
346,0.841559,Biquini Cavadão,"Mais um ano que se passa, mais um ano sem você. Já não tenho a mesma idade, envelheço na cidade. Essa vida é jogo rápido para mim ou prá você. Mais um ano que se passa e eu não sei o que fazer!. . Juventude se abraça, faz de tudo pra esquecer. Um feliz aniversário para mim ou prá você.... Feliz ..."
6380,0.813136,Ultraje A Rigor,"O chiclete que você mastiga não é igual ao meu. O chiclete que você mastiga não é igual ao meu. O chiclete que você mastiga não é igual ao meu. O chiclete que você mastiga não é igual ao meu. Enche a boca de ar. Depois dá um assoprão. Faz bem pro ""seu"" Mané do bar. Mas pra sua dentadura não. O c..."
6397,0.839995,Ultraje A Rigor,"Êta sonzinho fuleiro!. Êta sonzinho fuleiro!. Êta sonzinho fuleiro!. Êta sonzinho fuleiro!. é fuleiro. o sonzinho é fuleiro. yeah, what a fuler sound. yeah, what a fuler sound. yeah, what a fuler sound. yeah, what a fuler sound. é fuleiro. é fuleiro. ê paìsinho fuleiro. ê dinheirinho fuleiro. êt..."
6320,0.885689,Titãs,"Querem acabar comigo. Nem eu mesmo sei porque. Enquanto eu tiver você aqui. Ninguém poderá me destruir. Querem acabar comigo. Isso eu não vou deixar. Me abrace assim, me olhe assim. Não vá ficar, longe de mim. Pois enquanto eu tiver você comigo. Sou mais forte para mim não há perigo. Você está a..."
28,0.906091,Aliados,"Trocando o preconceito por um espelho,encontra o perfil,. O homem que mudar e sua imagem refletiu. Um lado desumano, de um insano tentava aparecer, mas perde as forças quando sabe o que é viver.. Sua imagem não faz diferença,pois somos iguais, corte os próprios punhos e verá, seu sangue é vermel..."
97,0.952168,Aliados,Quero te ligar. Mas não só pra dizer que eu errei. Tá ficando tarde. E o orgulho quer me destruir. Mas.... Não tem porque. É sempre assim. Falei sem pensar. Acredite em mim. O sangue ferveu. E a gente tretou. Fiquei fora do ar. Mas a raiva já passou. Tudo isso me fez repensar. Que pensando bem.....
1418,0.843334,Detonautas,"Quando eu me perco é quando eu te encontro. Quando eu me solto seus olhos me vêem. Quando eu me iludo é quando eu te esqueço. Quando eu te tenho eu me sinto tão bem. Você me fez sentir de novo o que eu. Já não me importava mais. Você me faz tão bem. Você me faz, você me faz tão bem. Quando eu te..."
208,0.874311,Barão Vermelho,o que você faz. depois do fim da festa. quando nada mais lhe resta. ninguém mais presta atenção. que que você faz. depois do fim da festa. a febre se manifesta. e o corpo implora proteção. que que você faz. quando um bar parece um barco. e a ciade um oceano. que que você faz. se nenhuma escolha ...
2358,0.894270,Ira!,Envelheço na cidade. Depois do solinho entra a batera e o baixo:. RIFF1. G F# Em G B A. G ----------------------------. D ----------------------------. A ------------------------2---. E 3-3-3-2-0-0---0-2-3-2-3---5-. RIFF2. C A C A A A A. G ---------------------------. D -------------------------...
59,0.990194,Aliados,"Se tem sede, chove assim. Temos águas pra afogar. Mas sobram mágoas pra beber. Eu sei que sim. Somos jovens pra sorrir. E sobram motivos pra sonhar. Não vamos chorar enquanto isso. E a gente vai ouvir. Mas só depois de reclamar. Queremos sim ou não queremos mais. A ilusão da perfeição. De viverm..."


Aqui podemos ver quão relacionadas as músicas de um artista ou banda estão com cada tópico

In [ ]:
# column names
topicnames = ["Topic {}".format(i + 1) for i in range(doc_topic_matrix.shape[1])]

letra_topico = pd.DataFrame(np.round(doc_topic_matrix, 2), columns=topicnames, index = letras.index)
letra_topico[['Artist', 'SName']] = letras[['Artist','SName']]

ordem = ['Artist', 'SName']
ordem.extend(topicnames)
letra_topico = letra_topico[ordem]

letra_topico.query('Artist == "Matanza"').sort_values(by = 'Topic 1', ascending = False).head()

,Artist,SName,Topic 1,Topic 2,Topic 3,Topic 4,Topic 5
3566,Matanza,Imbecil,0.69,0.01,0.18,0.01,0.13
3550,Matanza,Casa em Frente Ao Cemitério,0.52,0.37,0.00,0.10,0.00
3540,Matanza,A Casa em Frente ao Cemitério,0.52,0.37,0.00,0.10,0.00
3544,Matanza,As Melhores Putas do Alabama,0.48,0.30,0.11,0.10,0.01
3606,Matanza,Tudo Errado,0.48,0.22,0.00,0.00,0.30




*   Tópico 1:
   
    Tem como bandas: Aliados, Ira!, Ultraje a Rigor

*   List item





## Visualizando os tópicos

In [ ]:
!pip install pyLDAvis
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

In [ ]:
pyLDAvis.sklearn.prepare(lda, vec_text, tf_vectorizer, sort_topics=False, mds = 'tsne')

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)
/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:783: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:793: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:827: FutureWarning: 'square_distan

PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
0      150.222153  -72.269554       1        1  14.422243
1       51.163364 -159.968018       2        1  25.482944
2      -73.144554   21.523815       3        1  18.713098
3      -66.410133 -110.606438       4        1  18.085375
4       54.417324   12.443444       5        1  23.296340, topic_info=         Term         Freq        Total Category  logprob  loglift
70       amor  3303.000000  3303.000000  Default  30.0000  30.0000
1574      vai  4800.000000  4800.000000  Default  29.0000  29.0000
964   ninguém  1610.000000  1610.000000  Default  28.0000  28.0000
878       mim  2723.000000  2723.000000  Default  27.0000  27.0000
1250    quero  3529.000000  3529.000000  Default  26.0000  26.0000
...       ...          ...          ...      ...      ...      ...
456     dizer   504.616528  1265.057962   Topic5  -5.0210   0.5378
70       amor   752.557318  3303.673262   Topic5  -4.6213  -0.0224
155       bem   663.828384  2458.574688   Topic5  -4.7468   0.1476
513     então   486.454663  1221.222068   Topic5  -5.0577   0.5364
97       aqui   475.234691  2004.781476   Topic5  -5.0810   0.0174

[399 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
6         3  0.991494    abre
16        2  0.988831  acende
38        1  0.039014   agora
38        2  0.418718   agora
38        3  0.014836   agora
...     ...       ...     ...
1697      5  0.114764     vão
1700      1  0.991653    yeah
1700      4  0.006723    yeah
1701      4  0.997003     you
1702      4  0.989115    your

[835 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3, 4, 5])

In [ ]:
from sklearn.manifold import TSNE

letras_embedded = TSNE(n_components=2, verbose=1, perplexity=40, early_exaggeration=20).fit_transform(doc_topic_matrix)

/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:783: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:793: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  FutureWarning,


[t-SNE] Computing 121 nearest neighbors...
[t-SNE] Indexed 5277 samples in 0.005s...
[t-SNE] Computed neighbors for 5277 samples in 0.210s...
[t-SNE] Computed conditional probabilities for sample 1000 / 5277
[t-SNE] Computed conditional probabilities for sample 2000 / 5277
[t-SNE] Computed conditional probabilities for sample 3000 / 5277
[t-SNE] Computed conditional probabilities for sample 4000 / 5277
[t-SNE] Computed conditional probabilities for sample 5000 / 5277
[t-SNE] Computed conditional probabilities for sample 5277 / 5277
[t-SNE] Mean sigma: 0.075019
[t-SNE] KL divergence after 250 iterations with early exaggeration: 144.714798
[t-SNE] KL divergence after 1000 iterations: 1.202070


In [ ]:
letras = letras.assign(tsne1 = letras_embedded[:,0], tsne2 = letras_embedded[:,1])

alt.Chart(letras.sample(500)).mark_circle(
    opacity = .7,
    size = 30
).encode(
    x = 'tsne1',
    y = 'tsne2', 
    color = 'main_topic:N',
    size = 'main_topic_prob',
    tooltip = ['Artist', 'SName', 'main_topic', 'main_topic_prob']
).interactive()

alt.Chart(...)

In [ ]:
alt.Chart(letras.query('Artist == "Rita Lee"')).mark_circle(
    opacity = .7,
    size = 30
).encode(
    x = 'tsne1',
    y = 'tsne2', 
    color = 'main_topic:N',
    size = 'main_topic_prob',
    tooltip = ['Artist', 'SName', 'main_topic', 'main_topic_prob']
).interactive()

alt.Chart(...)